# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("output_data/cities.csv")
city_df.head()

,City,lat,long,Max Temp °C,Humidity %,Cloudiness %,Wind Speed,Country,Date
0,bethel,41.3712,-73.4140,17.19,81,0,1.54,US,1663385929
1,hithadhoo,-0.6000,73.0833,27.57,73,78,2.68,MV,1663385929
2,ziway,7.9333,38.7167,16.54,81,30,3.19,ET,1663385929
3,yellowknife,62.4560,-114.3525,10.31,87,40,3.60,CA,1663385929
4,muros,42.7762,-9.0603,17.31,82,100,3.54,ES,1663385930


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
#This contains a max_intesity that I felt could better convay collected data
locations = city_df[["lat", "long"]].astype(float)
humidity = city_df["Humidity %"].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:

#As per the the README.md provided descirbing the task I have added max humidity for the max_intensity
locations = city_df[["lat", "long"]].astype(float)
humidity = city_df["Humidity %"].astype(float)
humidity_max = humidity.max()
fig_max_humidity = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity= humidity_max,
                                 point_radius = 4)

fig_max_humidity.add_layer(heat_layer)

fig_max_humidity
#My dataset has a very high humidity


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
'''
A max temperature lower than 27 degrees but higher than 21.

Wind speed less than 4.5 m/s.

Zero cloudiness.
'''
#adusted values to meet the Module 6 Grading Rubric (10 or less results)
criteria_cities = city_df.loc[(city_df['Max Temp °C'] <= 27) & (city_df['Max Temp °C'] >= 24) & (city_df['Wind Speed'] < 4.5) & (city_df['Cloudiness %'] == 0)]
criteria_cities = criteria_cities.dropna()
criteria_cities

,City,lat,long,Max Temp °C,Humidity %,Cloudiness %,Wind Speed,Country,Date
79,benghazi,32.1167,20.0667,24.65,53,0,3.24,LY,1663385947
143,baghdad,33.3406,44.4009,24.95,44,0,3.60,IQ,1663385940
312,harrison,30.4091,-89.0436,26.03,88,0,2.06,US,1663386003
332,cayenne,4.9333,-52.3333,24.02,94,0,0.00,GF,1663386008
367,aswan,24.0934,32.9070,24.61,35,0,3.09,EG,1663385983
369,harqalah,36.0303,10.5090,24.92,73,0,2.57,TN,1663386019
513,sakakah,29.9697,40.2064,26.76,24,0,3.76,SA,1663386056


### Hotel Map

In [7]:
hotel_df = criteria_cities

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["long"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request 
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("No hotels found, skipping.")

In [8]:
hotel_df

,City,lat,long,Max Temp °C,Humidity %,Cloudiness %,Wind Speed,Country,Date,Hotel Name
79,benghazi,32.1167,20.0667,24.65,53,0,3.24,LY,1663385947,Tibesti Hotel
143,baghdad,33.3406,44.4009,24.95,44,0,3.60,IQ,1663385940,Baghdad Hotel
312,harrison,30.4091,-89.0436,26.03,88,0,2.06,US,1663386003,Residence Inn by Marriott Gulfport-Biloxi Airport
332,cayenne,4.9333,-52.3333,24.02,94,0,0.00,GF,1663386008,Hôtel Le Dronmi
367,aswan,24.0934,32.9070,24.61,35,0,3.09,EG,1663385983,Nuba Nile
369,harqalah,36.0303,10.5090,24.92,73,0,2.57,TN,1663386019,Dar Khadija
513,sakakah,29.9697,40.2064,26.76,24,0,3.76,SA,1663386056,Fakhamat Al Orjoana Apartment


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

In [11]:
# Add marker layer ontop of heat map
# Create a marker_layer using the poverty list to fill the info box

markers = gmaps.marker_layer(locations, hotel_info,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))